In [39]:
import xlrd
from xlutils.copy import copy

# 文件路径
path_r = 'HSI_日线数据.xls'
book = xlrd.open_workbook(path_r)
sheet = book.sheet_by_index(0)

# 获取行数和列数
nrows = sheet.nrows
ncols = sheet.ncols

# 读取数据
date = sheet.col_values(0)[1:]  # 日期，跳过标题
open_price = sheet.col_values(1)[1:]  # 开盘价，跳过标题
high = sheet.col_values(2)[1:]  # 最高价，跳过标题
low = sheet.col_values(3)[1:]  # 最低价，跳过标题
close = sheet.col_values(4)[1:]  # 收盘价，跳过标题
vol = sheet.col_values(5)[1:]  # 成交量，跳过标题
amount = sheet.col_values(6)[1:]  # 成交额，跳过标题


In [40]:
# 计算日涨跌
def calculate_daily_changes(close_prices):
    daily_changes = [0]  # 第一天没有涨跌，所以设为0
    for i in range(1, len(close_prices)):
        daily_change = float(close_prices[i]) - float(close_prices[i - 1])
        daily_changes.append(daily_change)
    return daily_changes

# 计算日涨跌幅
def calculate_daily_returns(close_prices):
    daily_returns = [0] # 第一天没有涨跌幅，所以设为0
    for i in range(1, len(close_prices)):
        prev_close = float(close_prices[i - 1])
        current_close = float(close_prices[i])
        daily_return = (current_close - prev_close) / prev_close if prev_close != 0 else 0
        daily_returns.append(daily_return)
    return daily_returns

# 股票10日涨幅
def Chg_ten(close):
    chg_ten = []
    for i in range(10, len(close)):
        c = (float(close[i]) - float(close[i - 10])) / float(close[i - 10])
        chg_ten.append(c)
    return chg_ten


def ADR(daily_changes):
    adr = []
    rise = 0
    fall = 0
    for i in range(len(daily_changes)):
        if i < 10:
            if daily_changes[i] > 0:
                rise += 1
            elif daily_changes[i] < 0:
                fall += 1
            if i == 9:
                adr_ratio = round(float(rise) / fall, 4) if fall != 0 else 0  # 替换 '*' 为 0
                adr.append(adr_ratio)
        else:
            if daily_changes[i - 10] > 0 and daily_changes[i] < 0:
                rise -= 1
                fall += 1
            elif daily_changes[i - 10] < 0 and daily_changes[i] > 0:
                rise += 1
                fall -= 1
            adr_ratio = round(float(rise) / fall, 4) if fall != 0 else 0  # 替换 '*' 为 0
            adr.append(adr_ratio)
    return adr



# 10日相对强弱指标(10日内涨的天数/10)
def RSI10(daily_changes):
    rsi_10 = []
    rise = 0
    for i in range(len(daily_changes)):
        if i < 10:
            if daily_changes[i] > 0:
                rise += 1
            if i == 9:
                rsi_10.append(rise / 10)
        else:
            if daily_changes[i - 10] > 0:
                rise -= 1
            if daily_changes[i] > 0:
                rise += 1
            rsi_10.append(rise / 10)
    return rsi_10



# 当日k线值=(PRICEclose−PRICEopen)/(PRICEhigh−PRICElow)
def k_value(high, low, open_price, close):
    k = []
    for i in range(len(open_price)):
        # 避免除以零
        if float(high[i]) != float(low[i]):
            kk = (float(close[i]) - float(open_price[i])) / (float(high[i]) - float(low[i]))
        else:
            # 如果最高价和最低价相同，决定如何处理
            # 这里我们可以将k值设为0，或者根据您的需求进行其他处理
            kk = 0
        k.append(kk)
    return k



# n日乖离率BIAS=（当日收盘价-N日内移动平均收盘价）/N日内移动平均收盘价
def BIAS(n, close):
    bias = []
    for i in range(n,len(close)):
        b = (close[i] - sum(close[i-n:i]) / n) / (sum(close[i-n:i]) / n)
        bias.append(b)
    return bias


# n日RSV-未成熟随机值:RSV=100x(当日收盘价–9日内最低收盘价)/(9日内最高收盘价–9日内最低收盘价)
def RSV(n, close):
    rsv = []
    for i in range(n,len(close)):
        r = (close[i] - min(close[i-n:i])) / (max(close[i-n:i]) - min(close[i-n:i]))
        rsv.append(10 * r)
    return rsv


# OBV=sign(当日股票收盘价−前日股票收盘价)×当日成交量V
def OBV(close, vol):
    obv = []
    for i in range(1, len(close)):
        x = close[i] - close[i - 1]
        if x == 0:
            sign = 0
        elif x > 0:
            sign = 1
        elif x < 0:
            sign = -1
        o = sign * vol[i]
        obv.append(o)
    return obv



In [41]:
#计算各个因子
daily_changes = calculate_daily_changes(close)
daily_returns = calculate_daily_returns(close)
k=k_value(high, low, open_price, close)
chg_ten=Chg_ten(close)
obv= OBV(close, vol)
adr=ADR(daily_changes)
rsi_10=RSI10(daily_changes)
bias=BIAS(12,close)
rsv=RSV(9,close)

l=len(k)


In [42]:
# 新建工作簿，并复制之前的表格
my_workbook = copy(book)
sheet = my_workbook.get_sheet(0)

In [43]:
for i in range(1, nrows): # 从1开始，跳过标题行
    sheet.write(i, ncols, daily_changes[i-1]) # 添加日涨跌到下一个空列
    sheet.write(i, ncols+1, daily_returns[i-1]) # 添加日涨跌幅到下一个空列

In [44]:
# 写入Excel表格
for i in range(l):
    # 因为已经添加了日涨跌和日涨跌幅，所以列号需要相应调整
    # 假设日涨跌和日涨跌幅分别写入第7列和第8列（索引从0开始，所以使用6和7）
    if i == 0:
        # 写入标题行
        sheet.write(i, 7, '日涨跌')
        sheet.write(i, 8, '日涨跌幅')
        sheet.write(i, 9, 'k值')
        sheet.write(i, 10, '10日涨幅')
        sheet.write(i, 11, 'OBV')
        sheet.write(i, 12, '10日涨跌比')
        sheet.write(i, 13, '10日相对强弱指标')
        sheet.write(i, 14, '12日乖离值')
        sheet.write(i, 15, '9日RSV')
    else:
        # 写入因子数据
        sheet.write(i, 7, daily_changes[i - 1])
        sheet.write(i, 8, daily_returns[i - 1])
        sheet.write(i, 9, k[i - 1] if i < len(k) else '')
        sheet.write(i, 10, chg_ten[i - 10] if i >= 10 else '')
        sheet.write(i, 11, obv[i - 1] if i < len(obv) else '')
        sheet.write(i, 12, adr[i - 10] if i >= 10 else '')
        sheet.write(i, 13, rsi_10[i - 10] if i >= 10 else '')
        sheet.write(i, 14, bias[i - 12] if i >= 12 else '')
        sheet.write(i, 15, rsv[i - 9] if i >= 9 else '')
    

In [45]:
# 保存文件
my_workbook.save('HSI_日线数据+factor.xls')